In [8]:
import pandas as pd
import numpy as np

In [9]:
ps = ['../callback_1757_0530_dnn_tl_result_0.0204_0.0236_mlp.csv']
print(ps)

['../callback_1757_0530_dnn_tl_result_0.0204_0.0236_mlp.csv']


In [10]:
if len(ps)>1:
    probs = []
    for p in ps:
        d = pd.read_csv(p)
        probs.append(d.proba.values)
    preds=np.mean(np.vstack(probs), axis=0)
    df = pd.DataFrame({'instanceID': range(1, len(preds)+1), 'proba': preds})
    df.to_csv('../ensembled.csv', index=False)

In [11]:
if len(ps)==1: # rule out users already installed
   pass
ui = pd.read_csv('../data/pre/user_installedapps.csv')
te = pd.read_csv('../data/pre/new_generated_test.csv')

In [21]:
ui_list = ui.groupby('userID').apply(lambda df: list(df.appID.values)).reset_index()
    
ui_list.columns = ['userID', 'insApps']

In [22]:
preds = pd.read_csv(ps[0])['proba'].values

In [23]:
adApp = te.appID.values
adApp

array([ 14,  14, 465, ..., 328, 465, 465])

In [24]:
userID = te.userID.values

In [25]:
ui_list.head()

,userID,insApps
0,1,"[357, 360, 362, 365, 375, 441, 480, 481, 482, ..."
1,9,"[538, 602, 604]"
2,10,"[357, 359, 365, 383, 480, 481, 482, 483, 484, ..."
3,12,"[480, 481, 482, 483, 484, 485, 486, 487, 488, ..."
4,14,"[362, 480, 481, 482, 483, 484, 485, 486, 487, ..."


In [ ]:
ui_dic = ui_list.set_index('userID').to_dict()['insApps']

In [ ]:
c=0
for i,uid in enumerate(preds):
    if userID[i] in ui_dic:
        if adApp[i] in ui_dic[userID[i]]:
            preds[i] = 0
            c += 1
print('Finished ruling ', c)

Finished ruling  2258


In [ ]:
import pickle
pickle.dump((ui_dic, adApp, userID), open('../data/pre/dump_rule.bin', 'wb'))

In [ ]:
(ui_dic, adApp, userID) = pickle.load('../data/pre/dump_rule.bin', 'rb')

In [ ]:
ui_dic